In [1]:
#import dependencies
import requests
import pandas as pd

# SQL Alchemy
from sqlalchemy import create_engine, event

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_user, remote_db_pwd, remote_db_endpoint, remote_db_port, remote_db_name

In [10]:
# read in csv as df
super_bowl_df = pd.read_csv('superbowl_table.csv')
del super_bowl_df['Unnamed: 0']
del super_bowl_df['Unnamed: 14']
del super_bowl_df['Unnamed: 15']
del super_bowl_df['Unnamed: 16']
super_bowl_df.head()

,Year,Season,SB,Winner,Winner_code,Winner Pts,Loser,Loser_code,Loser Pts,MVP,Stadium,City,State
0,2021,2020,LV (55),Tampa Bay Buccaneers,TB,31,Kansas City Chiefs,KC,9,Tom Brady,Raymond James Stadium,Tampa,Florida
1,2020,2019,LIV (54),Kansas City Chiefs,KC,31,San Francisco 49ers,SF,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida
2,2019,2018,LIII (53),New England Patriots,NE,13,Los Angeles Rams,LAR,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia
3,2018,2017,LII (52),Philadelphia Eagles,PHI,41,New England Patriots,NE,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
4,2017,2016,LI (51),New England Patriots,NE,34,Atlanta Falcons,ATL,28,Tom Brady,NRG Stadium,Houston,Texas


In [11]:
# Create Engine and Pass in MySQL Connection

# Remote MySQL DB Connection  
remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
remote_conn = remote_engine.connect()

# Create remote database
try:
    remote_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {remote_db_name}")
except Exception as e:
    print(e)

# Connect to remote database
try:
    remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
    remote_conn = remote_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
remote_engine.table_names()

(pymysql.err.ProgrammingError) (1007, "Can't create database 'fantasy_project'; database exists")
[SQL: CREATE DATABASE fantasy_project]
(Background on this error at: http://sqlalche.me/e/13/f405)


['fantasy_stats', 'super_bowl_stats']

In [12]:
#upload dataframe to database
super_bowl_df.to_sql(name='super_bowl_stats', if_exists='replace', con=remote_conn, index=False)
remote_engine.table_names()

['fantasy_stats', 'super_bowl_stats']